### 스타벅스 매장 입지 분석
- 순서
    1. 데이터 수집
    2. 데이터 전처리
    3. 분석
    4. 시각화

#### 데이터 수집
- 셀레니움 자동화 + 뷰티플수프 정제

In [1]:
# 필요 라이브러리 사용
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

In [99]:
!pip install tqdm

In [100]:
# 분석, 시각화 라이브러리 사용
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams, rc, font_manager
import warnings
import seaborn as sns
import time
from tqdm import tqdm  # 반복 진행 프로그레스바 모듈

In [6]:
# 맷플롯립 한글 및 기타설정
## 맑은 고딕
font_path = 'C:/Windows/Fonts/malgun.ttf'
font_name = font_manager.FontProperties(fname=font_path).get_name() # Malgun Gothic
warnings.simplefilter('ignore') # 경고메시지 출력숨김

## 맷플롯립 설정
plt.rcParams['font.family'] = font_name # 폰트 설정
plt.rcParams['font.size'] = 12 # 글자크기
plt.rcParams['figure.figsize'] = (12, 6) # (W, H)
plt.rcParams['axes.grid'] = True # 차트 가로세로줄 표시
plt.rcParams['axes.unicode_minus'] = False # 한글설정 후 마이너스깨짐 방지

## 시본 설정
sns.set_theme(font=font_name, style='darkgrid', rc={'axes.unicode_minus':False})

##### 셀레니움 사용

In [45]:
# 웹드라이버
driver = webdriver.Chrome()

In [49]:
# 스타벅스 코리아사이트 연결
url = 'https://www.starbucks.co.kr/store/store_map.do?disp=quick'
driver.get(url)

In [50]:
# 매장찾기 클릭 
## header.find_store_header > h2.btn_find_store > a
link_path = 'header.find_store_header > h2.btn_find_store > a'
driver.find_element(By.CSS_SELECTOR, link_path).click()

In [51]:
# 매장찾기 웹소스에서 '지역검색' 링크 검색 후 링크 클릭
## F12 개발자도구로 HTML 태그 정보 확인
## header.loca_search > h3 > a 에 지역검색 링크 존재
link_path = 'header.loca_search > h3 > a'
driver.find_element(By.CSS_SELECTOR, link_path).click()

In [52]:
# 지역검색에서 서울 링크 클릭
## ul.sido_arae_box > li:nth-child(1) > a
link_path = 'ul.sido_arae_box > li:nth-child(1) > a'
driver.find_element(By.CSS_SELECTOR, link_path).click()

In [53]:
# 서울에서 전체 링크 클릭
## ul.gugun_arae_box > li:nth-child(1) > a
link_path = 'ul.gugun_arae_box > li:nth-child(1) > a'
driver.find_element(By.CSS_SELECTOR, link_path).click()
# 매장 전체 조회가 최초 2~3초 정도 시간이 걸리기 때문에 딜레이를 걸어줌
time.sleep(5.0)

- 메가커피, 빽다방, 컴포즈커피 등 크롤링 가능한 사이트도 동일하게 진행

##### 서울 스타벅스 매장 정보 가져오기


In [54]:
html2 = driver.page_source

In [56]:
soup = BeautifulSoup(html2, 'html.parser')

In [57]:
# 서울, 전체 결과에서
## ul.quickSearchResultBoxSidoGugun > li.quickResultLstCon 만 가져오면 됨
quickResultLst = soup.select('li.quickResultLstCon')
len(quickResultLst)

637

In [62]:
# 각 결과 확인, 부산 현재위치에서 나온 23개 결과가 서울과 중첩
## 판다스 데이터프레임에서 전처리해주면 끝!
quickResultLst[23]

<li class="quickResultLstCon" data-code="3762" data-hlytag="null" data-index="0" data-lat="37.501087" data-long="127.043069" data-name="역삼아레나빌딩" data-storecd="1509" style="background:#fff"> <strong data-my_siren_order_store_yn="N" data-name="역삼아레나빌딩" data-store="1509" data-yn="N">역삼아레나빌딩  </strong> <p class="result_details">서울특별시 강남구 언주로 425 (역삼동)<br/>1522-3232</p> <i class="pin_general">리저브 매장 2번</i></li>

##### 판다스 데이터처리

In [96]:
quickResultLst[31].select('i')[0]['class'][0].split('_')[1]

'reserve'

In [104]:
# 서울매장 리스트 DF
seoulStores = []

for item in tqdm(quickResultLst): # 진행률 표시
    storeName = item.select('strong')[0].text.strip() # 매장이름
    storeLat = item['data-lat'] # 매장위치 위도값
    storeLng = item['data-long'] # 매장위치 경도값
    address = item.select('p.result_details')[0].text.replace('1522-3232', '') # 매장 주소
    storeCd = item['data-storecd'] # 매장코드(번호가 작을수록 오래된 매장이라고 유추가능)
    storeType = item.select('i')[0]['class'][0].split('_')[1] # 매장종류(일반,리저브,DT,WT)

    seoulStores.append([storeCd, storeName, storeType, address, storeLat, storeLng])
    # time.sleep(0.1) # 일부러 딜레이 걸지마세요


100%|██████████| 637/637 [00:00<00:00, 5388.42it/s]


In [106]:
# 리스트 확인
len(seoulStores)

637

In [111]:
# DF로 변환
df_seoulStore = pd.DataFrame(seoulStores, columns=['매장코드','매장명','매장종류','주소','위도','경도'])
df_seoulStore.head(2)

,매장코드,매장명,매장종류,주소,위도,경도
0,1190,동명대DT,generalDT,부산광역시 남구 신선로 423 (용당동),35.12311959047579,129.09901642703608
1,2235,부산유엔공원,general,부산광역시 남구 유엔로 200 (대연동),35.1299808,129.0980971


In [112]:
df_seoulStore.tail(3)

,매장코드,매장명,매장종류,주소,위도,경도
634,1668,묵동,general,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",37.615368,127.076633
635,2002,양원역,general,서울특별시 중랑구 양원역로10길 3 (망우동),37.6066536267232,127.106359790053
636,1749,중화역,general,서울특별시 중랑구 봉화산로 35 1층,37.60170912407773,127.07841136432036


In [113]:
# 데이터 저장
df_seoulStore.to_csv('./data/스타벅스_서울매장정보.csv', encoding='utf-8')

##### 데이터 전처리
- 데이터로 통계를 낼때 문제가 없도록 데이터를 전부 사전에 처리를 해주는 것
    - 필요없는 행,컬럼 제거
    - 필요한데 없는 행,컬럼 추가
    - 잘못된 값 변경
    - 데이터 결측치(수치값이 빠진것) 처리
        - Null, NaN, '', empty 등 값이 없는 컬럼을 제거/유용한 값으로 치환
        - Null, NaN은 통계 계산하면 계산된 값도 Null 로 바꿔버림
        - 숫자로 된 컬럼값에는 결측치를 무조건 제거/변경 해줘야 함

- 데이터 전처리가 빅데이터 분석 전체 프로젝트에 거의 50% 차지함

In [114]:
# 재확인
df_seoulStore

,매장코드,매장명,매장종류,주소,위도,경도
0,1190,동명대DT,generalDT,부산광역시 남구 신선로 423 (용당동),35.12311959047579,129.09901642703608
1,2235,부산유엔공원,general,부산광역시 남구 유엔로 200 (대연동),35.1299808,129.0980971
2,683,부산대연역,general,"부산광역시 남구 수영로 240-1, 1층 (대연동)",35.134998534278104,129.0930603381513
3,1795,부산대연못골,general,부산광역시 남구 못골로 87 (대연동),35.13609516236527,129.09191736599408
4,248,경성대,general,부산광역시 남구 수영로 312 (대연동),35.137345553736964,129.10063775537583
...,...,...,...,...,...,...
632,838,사가정역,general,서울특별시 중랑구 면목로 310,37.579594,127.087966
633,493,상봉역,general,서울특별시 중랑구 망우로 307 (상봉동),37.59689,127.08647
634,1668,묵동,general,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",37.615368,127.076633
635,2002,양원역,general,서울특별시 중랑구 양원역로10길 3 (망우동),37.6066536267232,127.106359790053


In [117]:
# 전체 데이터에서 주소에 부산광역시가 포함된 여부 출력
df_seoulStore['주소'].str.contains('부산광역시')

0       True
1       True
2       True
3       True
4       True
       ...  
632    False
633    False
634    False
635    False
636    False
Name: 주소, Length: 637, dtype: bool

In [119]:
# 부산광역시가 포함된 인덱스 출력
del_list = df_seoulStore[df_seoulStore['주소'].str.contains('부산광역시')].index

In [120]:
# 서울이 아닌 인덱스 값을 삭제(drop)
df_seoulStore.drop(del_list)

,매장코드,매장명,매장종류,주소,위도,경도
23,1509,역삼아레나빌딩,general,서울특별시 강남구 언주로 425 (역삼동),37.501087,127.043069
24,1434,논현역사거리,general,서울특별시 강남구 강남대로 538 (논현동),37.510178,127.022223
25,1595,신사역성일빌딩,general,서울특별시 강남구 강남대로 584 (논현동),37.5139309,127.0206057
26,1527,국기원사거리,general,서울특별시 강남구 테헤란로 125 (역삼동),37.499517,127.031495
27,1468,대치재경빌딩,general,서울특별시 강남구 남부순환로 2947 (대치동),37.494668,127.062583
...,...,...,...,...,...,...
632,838,사가정역,general,서울특별시 중랑구 면목로 310,37.579594,127.087966
633,493,상봉역,general,서울특별시 중랑구 망우로 307 (상봉동),37.59689,127.08647
634,1668,묵동,general,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",37.615368,127.076633
635,2002,양원역,general,서울특별시 중랑구 양원역로10길 3 (망우동),37.6066536267232,127.106359790053


- drop() 등 몇가지 함수는 값을 적용하는게 아니라 값이 바뀐걸 보여주는 함수
- 값을 변경하려면, 새로운 변수나 같은 변수에 다시 할당하거나, 속성중 inplace=True로 실행하면 됨

In [123]:
# 1. drop() 함수로 변경된 값을 재할당
df_seoulStore2 = df_seoulStore.drop(del_list)
df_seoulStore2

,매장코드,매장명,매장종류,주소,위도,경도
23,1509,역삼아레나빌딩,general,서울특별시 강남구 언주로 425 (역삼동),37.501087,127.043069
24,1434,논현역사거리,general,서울특별시 강남구 강남대로 538 (논현동),37.510178,127.022223
25,1595,신사역성일빌딩,general,서울특별시 강남구 강남대로 584 (논현동),37.5139309,127.0206057
26,1527,국기원사거리,general,서울특별시 강남구 테헤란로 125 (역삼동),37.499517,127.031495
27,1468,대치재경빌딩,general,서울특별시 강남구 남부순환로 2947 (대치동),37.494668,127.062583
...,...,...,...,...,...,...
632,838,사가정역,general,서울특별시 중랑구 면목로 310,37.579594,127.087966
633,493,상봉역,general,서울특별시 중랑구 망우로 307 (상봉동),37.59689,127.08647
634,1668,묵동,general,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",37.615368,127.076633
635,2002,양원역,general,서울특별시 중랑구 양원역로10길 3 (망우동),37.6066536267232,127.106359790053


In [126]:
# 2. inplace=True 사용
df_seoulStore.drop(del_list, inplace=True)

In [127]:
df_seoulStore

,매장코드,매장명,매장종류,주소,위도,경도
23,1509,역삼아레나빌딩,general,서울특별시 강남구 언주로 425 (역삼동),37.501087,127.043069
24,1434,논현역사거리,general,서울특별시 강남구 강남대로 538 (논현동),37.510178,127.022223
25,1595,신사역성일빌딩,general,서울특별시 강남구 강남대로 584 (논현동),37.5139309,127.0206057
26,1527,국기원사거리,general,서울특별시 강남구 테헤란로 125 (역삼동),37.499517,127.031495
27,1468,대치재경빌딩,general,서울특별시 강남구 남부순환로 2947 (대치동),37.494668,127.062583
...,...,...,...,...,...,...
632,838,사가정역,general,서울특별시 중랑구 면목로 310,37.579594,127.087966
633,493,상봉역,general,서울특별시 중랑구 망우로 307 (상봉동),37.59689,127.08647
634,1668,묵동,general,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",37.615368,127.076633
635,2002,양원역,general,서울특별시 중랑구 양원역로10길 3 (망우동),37.6066536267232,127.106359790053


In [130]:
# DF 인덱스가 0이 아닌값으로 시작, 인덱스 초기화
## drop=True는 인덱스 삭제, inplace=True 값 변경 완전적용(like Commit)
df_seoulStore.reset_index(inplace=True)

In [136]:
# 단순히 컬럼을 삭제할 때, 
## axis=0(기본, 삭제해도 됨) 행삭제, axis=1 컬럼
df_seoulStore.drop(['index'], axis=1, inplace=True)

In [137]:
df_seoulStore

,매장코드,매장명,매장종류,주소,위도,경도
0,1509,역삼아레나빌딩,general,서울특별시 강남구 언주로 425 (역삼동),37.501087,127.043069
1,1434,논현역사거리,general,서울특별시 강남구 강남대로 538 (논현동),37.510178,127.022223
2,1595,신사역성일빌딩,general,서울특별시 강남구 강남대로 584 (논현동),37.5139309,127.0206057
3,1527,국기원사거리,general,서울특별시 강남구 테헤란로 125 (역삼동),37.499517,127.031495
4,1468,대치재경빌딩,general,서울특별시 강남구 남부순환로 2947 (대치동),37.494668,127.062583
...,...,...,...,...,...,...
609,838,사가정역,general,서울특별시 중랑구 면목로 310,37.579594,127.087966
610,493,상봉역,general,서울특별시 중랑구 망우로 307 (상봉동),37.59689,127.08647
611,1668,묵동,general,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",37.615368,127.076633
612,2002,양원역,general,서울특별시 중랑구 양원역로10길 3 (망우동),37.6066536267232,127.106359790053


In [139]:
## 전처리 완료한 DF 저장
df_seoulStore.to_csv('./data/스타벅스_서울매장정보_최종.csv')

##### 전국 시군구 위경도에서 서울만 추출

In [140]:
# 전국 시군구 위경도 엑셀파일 로드
df_koreaManicipality = pd.read_excel('./data/전국_시군구_위경도.xlsx')

In [142]:
df_koreaManicipality.tail()

,docity,do,city,longitude,latitude
290,충청충주시,충청,충주시,127.928144,36.988181
291,충청태안군,충청,태안군,126.299975,36.742667
292,충청한누리대로,충청,한누리대로,127.289926,36.485450
293,충청홍성군,충청,홍성군,126.662908,36.598361
294,충청대전시,충청,대전시,127.384862,36.350630


In [144]:
# 서울구별 데이터 필터링
seoulList = df_koreaManicipality['do'] == '서울'

In [147]:
df_seoulManicipality = df_koreaManicipality[seoulList]

In [149]:
df_seoulManicipality.reset_index(drop=True, inplace=True)

In [151]:
df_seoulManicipality.tail()

,docity,do,city,longitude,latitude
20,서울용산구,서울,용산구,126.967522,37.536094
21,서울은평구,서울,은평구,126.931242,37.599969
22,서울종로구,서울,종로구,126.981642,37.570378
23,서울중구,서울,중구,126.999642,37.561003
24,서울중랑구,서울,중랑구,127.094778,37.603806


In [152]:
# 서울 구별 위치값 저장
df_seoulManicipality.to_csv('./data/서울구별위치.csv', encoding='utf-8')

In [154]:
# DF 정보확인 / 데이터 결측치, 타입 확인
df_seoulStore.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   매장코드    614 non-null    object
 1   매장명     614 non-null    object
 2   매장종류    614 non-null    object
 3   주소      614 non-null    object
 4   위도      614 non-null    object
 5   경도      614 non-null    object
dtypes: object(6)
memory usage: 28.9+ KB


In [155]:
df_seoulManicipality.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   docity     25 non-null     object 
 1   do         25 non-null     object 
 2   city       25 non-null     object 
 3   longitude  25 non-null     float64
 4   latitude   25 non-null     float64
dtypes: float64(2), object(3)
memory usage: 1.1+ KB
